## Что нужно было сделать
1. Написать Dataset для задачи seq2seq
2. Реализовать модель
3. Сделать цикл обучения
4. Реализовать метод генерации ответа по вопросу с помощью вашей модели

## Что было сделано (отмечено ✓)

1. Сделать модель, основанную на lstm/gru 5 баллов ✓
2. Сделать модель, основанную на cnn 7 баллов
3. Сделать модель, основанную на трансформере (реализовать все слои самому) 10 баллов
4. Добавить в rnn/cnn модель attention 5 баллов ✓
5. Реализовать жадное семплирование (генерацию по самому вероятному токену, как выше в языковой модели) 3 балла ✓
6. Реализовать beam search 5 баллов
7. Реализовать nucleus sampling 5 баллов ✓
8. Добавить condition в модель 3 балла
9. Добавить layer norm/residual в cnn или rnn модель 1 балл ✓
10. Реализовать аккамуляцию градиентов 1 балл
11. Сделать телеграм бота 2 балла
Конвертируем json в более удобный формат.

## Шаги решения

`1.` Сначала я преобразовал данные в более удобный формат: файлы `source` и
`target` содержат пары вопросов и ответов на них.
```
python convert_data.py
```
`2.` Затем данные были разделены на train, dev, test.
```
python split_data.py data/source data/target data 42
```
`3.` Обучил модель и нагенерировал ответы с помощью **greedy search**
и **nucleus sampling** (файлы `greedy_responses.txt` и `nucleus_responses.txt` в
папке `results`).
```
python train.py
```

Ниже процесс обучения для наглядности.

In [1]:
from src.data import basic_load
from src.tokenization import train_bpe, batch_tokenize
from src.datasets import QAData
from src.models import MyNet
from src.training import training_cycle
from src.generation import Generator
from torch.utils.data import DataLoader
import torch
import youtokentome as yttm


SOURCE_TRAIN_PATH = 'data/source.train'
SOURCE_DEV_PATH = 'data/source.dev'
SOURCE_TEST_PATH = 'data/source.test'
TARGET_TRAIN_PATH = 'data/target.train'
TARGET_DEV_PATH = 'data/target.dev'
TARGET_TEST_PATH = 'data/target.test'
BPE_TEXT_PATH = 'models/bpe_raw.txt'
BPE_MODEL_PATH = 'models/bpe_qa.model'
RESPONSES_GREEDY_PATH = 'results/greedy_responses.txt'
RESPONSES_NUCLEUS_PATH = 'results/nucleus_responses.txt'
VOCAB_SIZE = 7000
MAX_SOURCE_LEN = 40
MAX_TARGET_LEN = 40
PAD_INDEX = 0
UNK_INDEX = 1
UNK_INDEX = 2
EOS_INDEX = 3
BATCH_SIZE = 512
EMB_DIM = 512
HIDDEN_SIZE = 512


TRAIN_BPE = True
TRAIN_NET = True
GENERATE = True

source_train = basic_load(SOURCE_TRAIN_PATH)
target_train = basic_load(TARGET_TRAIN_PATH)
source_dev = basic_load(SOURCE_DEV_PATH)
target_dev = basic_load(TARGET_DEV_PATH)

assert len(source_train) == len(target_train)
assert len(source_dev) == len(target_dev)

if TRAIN_BPE:

    train_bpe(sentences=source_train,
              bpe_text_path=BPE_TEXT_PATH,
              bpe_model_path=BPE_MODEL_PATH,
              vocab_size=VOCAB_SIZE)

bpe = yttm.BPE(model=BPE_MODEL_PATH)

source_train_tokenized = batch_tokenize(source_train, bpe, bos=False, eos=False)
source_dev_tokenized = batch_tokenize(source_dev, bpe, bos=False, eos=False)
target_train_tokenized = batch_tokenize(target_train, bpe, bos=False, eos=False)
target_dev_tokenized = batch_tokenize(target_dev, bpe, bos=False, eos=False)

assert len(source_train_tokenized) == len(target_train_tokenized)
assert len(source_dev_tokenized) == len(target_dev_tokenized)

train_ds = QAData(source_train_tokenized,
                  target_train_tokenized,
                  MAX_SOURCE_LEN,
                  MAX_TARGET_LEN)

valid_ds = QAData(source_dev_tokenized,
                  target_dev_tokenized,
                  MAX_SOURCE_LEN,
                  MAX_TARGET_LEN)

train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_ds, BATCH_SIZE)

GPU = torch.cuda.is_available()

if GPU:
    print('Using GPU...')
    device = torch.device('cuda')
else:
    raise NotImplementedError

model = MyNet(emb_dim=EMB_DIM,
              hidden_size=HIDDEN_SIZE,
              vocab_size=VOCAB_SIZE,
              dropout=0.4,
              weight_tying=True)

model.to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_INDEX)
optimizer = torch.optim.Adam(params=model.parameters())

if TRAIN_NET:

    training_cycle(model, train_loader, valid_loader, optimizer, criterion,
                   device,  3., 5)


if GENERATE:

    source_test = basic_load(SOURCE_TEST_PATH)[:100]
    target_test = basic_load(TARGET_TEST_PATH)[:100]

    model.load_state_dict(torch.load('models/best_language_model_state_dict.pth'))

    generator = Generator(bpe, model, device)
    generator.to_file(source_test, target_test, RESPONSES_GREEDY_PATH, 'greedy')
    generator.to_file(source_test, target_test, RESPONSES_NUCLEUS_PATH, 'nucleus')





Attempting to create temporary data...
Temporary data successfully created!
BPE model successfully trained!
Attempting to remove temporary data...
Temporary data successfully removed!
Using GPU...
Epoch: 1
Train: loss - 6.6433 | perplexity - 767.626
Validation: loss - 5.9259 | perplexity - 374.604
Epoch: 2
Train: loss - 5.8663 | perplexity - 352.942
Validation: loss - 5.6647 | perplexity - 288.488
Epoch: 3
Train: loss - 5.6290 | perplexity - 278.373
Validation: loss - 5.6384 | perplexity - 281.014
Epoch: 4
Train: loss - 5.4651 | perplexity - 236.294
Validation: loss - 5.4313 | perplexity - 228.448
Epoch: 5
Train: loss - 5.3401 | perplexity - 208.530
Validation: loss - 5.3402 | perplexity - 208.554


Generating responses using nucleus search...: 100%|██████████| 100/100 [00:09<00:00, 10.43it/s]


#### Greedy Search

В целом, выбирая наиболее вероятный токен, можно получить какой-то относительно вменяемый
(с учетом источника) и может быть даже реалистичный ответ. Ниже привел несколько вопросов
и ответов модели на них.

```
Q: счастье - что ето

A: счастье это счастье!
```

```
Q: вопрос знатокам музыки.

A: [ссылка заблокирована по решению администрации проекта]
```

```
Q: сколько дней я работаю на вазе?

A: 15 минут 20 минут 10 дней
```

```
Q: может надо было?

A: может быть и не надо.
```

```
Q: посоветуйте ужасы на подобии у холмов есть глаза и зеркала

A: у холла в глазах и глазах
```
#### Nucleus Sampling

Случайный выбор из подвыборки токенов, суммарная вероятность которых больше `p`, к сожалению
даёт не очень хорошие результаты. Полагаю, что использовать nucleus sampling имеет смысл
вместе с другими методами.

Ниже приведены примеры ответов модели.
```
Q: если официант в ресторане что-то не то вам принёс или сделал что вы будете делать???

A:  сколько снятно понимать, сказать тяжелее делают сказал плохо делать? если достаточно
    определяет похвалиных образом насньняще привычки, предотщетрена
```
```
Q: счастье - что ето

A: нет жаром которые трядонемые (())))
```


```
Q: какие чулки мне одеть, чтоб теплые тока были? подскажите а?

A:  японская керора. дорогих ходы общных тупых моденной востоки
    главное мешком можете макси stsel nad hiki?
```


```
Q: а вы злые или добрые с утра, м?

A:  тут породы вам?), подобный телефон. точно социальнощей работодателяуру,
    зараженую помочь обществоку))
```